# Задача мэтчинга товаров

**Цели исследования:**  
  
Для каждого товара магазина найти один или несколько объектов из ассортимента магазина-конкурента, которые близки к нему по некоторой заданной метрике. 

**Задача:**
  
Разработать модель для метчинга товаров в соответствии с требуемой метрикой.
- разработать алгоритм, который для всех товаров магазина предложит несколько вариантов наиболее похожих товаров из ассортимента магазина-конкурента;
- оценить качество алгоритма по метрике accuracy@5

  
**План работы:**
  
- загрузить и изучить представленные данные;
- провести необходимую предобработку данных;
- провести исследовательский анализ данных;
- провести корреляционный анализ признаков, сделать выводы о мультиколлинеарности и при необходимости устранить её.
- выполнить подготовку признаков в пайплайне;
- выбрать лучшую модель и проверить её качество;
- провести анализ важности признаков, сделать выводы об их значимости;
- сформировать выводы и рекомендации по каждому шагу исследования;
- сформировать общий вывод и рекомендации.

**Какими данными располагаем:** 
  
- `base.csv` - анонимизированный набор товаров. Каждый товар представлен как уникальный id (0-base, 1-base, 2-base) и вектор признаков размерностью 72.
- `train.csv` - обучающий датасет. Каждая строчка - один товар, для которого известен уникальный id (0-query, 1-query, …) , вектор признаков И id товара из base.csv, который максимально похож на него (по мнению экспертов).
- `validation.csv` - датасет с товарами (уникальный id и вектор признаков), для которых надо найти наиболее близкие товары из base.csv
- `validation_answer.csv` - правильные ответы к предыдущему файлу.

In [ ]:
FAISS
Annoy
Qdrant

In [18]:
%%capture

# Стандартные библиотеки
import os
import re
import sys
import time
import warnings
from datetime import datetime
from math import ceil

# Апдейт и установка необходимых пакетов
!"{sys.executable}" -m pip install -U numba
!"{sys.executable}" -m pip install numpy==1.26.4
!"{sys.executable}" -m pip install scipy==1.13.1
!"{sys.executable}" -m pip install pandas==1.4.4
!"{sys.executable}" -m pip install --upgrade scikit-learn
!"{sys.executable}" -m pip install --upgrade matplotlib
!"{sys.executable}" -m pip install --upgrade seaborn
!"{sys.executable}" -m pip install --upgrade jinja2==3.1.4
!"{sys.executable}" -m pip install catboost
!"{sys.executable}" -m pip install missingno
!"{sys.executable}" -m pip install phik
!"{sys.executable}" -m pip install shap
!"{sys.executable}" -m pip install tqdm 

# Сторонние библиотеки
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik
import seaborn as sns
import shap
from IPython.display import display, HTML
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from matplotlib.ticker import MultipleLocator
import missingno as msno
from pandas.plotting import register_matplotlib_converters
from scipy import stats as st

# # Библиотеки scikit-learn
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.compose import ColumnTransformer
# from sklearn.dummy import DummyRegressor
# from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
# from sklearn.experimental import enable_halving_search_cv
# from sklearn.impute import SimpleImputer
# from sklearn.inspection import permutation_importance
# from sklearn.linear_model import LinearRegression, Lasso, Ridge
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
# (
#     GridSearchCV, RandomizedSearchCV, HalvingGridSearchCV, train_test_split
# )
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import (
#     LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder, RobustScaler, StandardScaler
# )
# from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle

# FAISS
import faiss

# Дополнительные библиотеки
from tqdm import tqdm

# Дополнительные настройки
matplotlib_axes_logger.setLevel('ERROR')
warnings.filterwarnings("ignore")
warnings.warn("ignore")
register_matplotlib_converters()

# Зафиксированные параметры визуализации
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
sns.set(rc={'figure.figsize': (20, 10)})
mpl.rcParams.update({'font.size': 11})
sns.set_style("whitegrid")

In [2]:
def unload_df(file_name, parse_dates=None, sep=None, dec=','):
    """
    Ищет файл в сети и локально, загружает его и возвращает как pandas DataFrame.
    Также умеет парсить дату по столбцам.

    :param file_name: имя файла для загрузки
    :param sep: разделитель колонок в файле (например, ',' для CSV)
    :param dec: символ десятичного разделителя (по умолчанию ',')
    :param parse_dates: список столбцов, которые нужно разобрать как даты
    :return: загруженный DataFrame или None, если файл не найден
    """
    if parse_dates is None:
        parse_dates = []
    file_path_net = f'/datasets/{file_name}'
    file_path_local = file_name

    try:
        if os.path.exists(file_path_net):
            file_path = file_path_net
        elif os.path.exists(file_path_local):
            file_path = file_path_local
        else:
            print(f'{file_name} не найден нигде')
            return None
        
        df = pd.read_csv(file_path, parse_dates=parse_dates, sep=sep, decimal=dec)
        location = "сети" if file_path == file_path_net else "локального хранилища"
        print(f'{file_name} успешно загружен из {location}')
        return df
    except Exception as e:
        print(f'Произошла ошибка при загрузке: {e}')
        return None
    
def look_on(df):
    display(df.head())
    df.info()

In [6]:
base = unload_df('./sample/base.csv')
train = unload_df('./sample/train.csv')
validation_answer = unload_df('./sample/validation_answer.csv')
validation = unload_df('./sample/validation.csv')

./sample/base.csv успешно загружен из локального хранилища
./sample/train.csv успешно загружен из локального хранилища
./sample/validation_answer.csv успешно загружен из локального хранилища
./sample/validation.csv успешно загружен из локального хранилища


In [7]:
look_on(base)

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,4207931-base,-43.946243,15.364378,17.515854,-132.31146,157.064420,-4.069252,-340.630860,-57.550140,128.398220,...,-71.927170,30.711966,-90.190475,-24.931271,66.972534,106.346634,-44.270622,155.98834,-1074.464888,-25.066608
1,2710972-base,-73.004890,4.923342,-19.750746,-136.52908,99.907170,-70.709110,-567.401996,-128.890150,109.914986,...,-109.044660,20.916021,-171.201390,-110.596844,67.730100,8.909615,-9.470253,133.29536,-545.897014,-72.913230
2,1371460-base,-85.565570,-0.493598,-48.374817,-157.98502,96.809510,-81.710210,-22.297688,79.768670,124.357086,...,-58.821650,41.369606,-132.934500,-43.016839,67.871925,141.778240,69.048520,111.72038,-1111.038833,-23.087206
3,3438601-base,-105.564090,15.393871,-46.223934,-158.11488,79.514114,-48.944480,-93.713010,38.581398,123.397960,...,-87.907290,-58.806870,-147.794800,-155.830237,68.974754,21.397510,126.098785,139.73320,-1282.707248,-74.527940
4,422798-base,-74.638880,11.315012,-40.204174,-161.76430,50.507114,-80.775560,-640.923467,65.225000,122.344940,...,-30.002094,53.642930,-149.823230,176.921371,69.473280,-43.395180,-58.947716,133.84064,-1074.464888,-1.164146


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291813 entries, 0 to 291812
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Id      291813 non-null  object 
 1   0       291813 non-null  float64
 2   1       291813 non-null  float64
 3   2       291813 non-null  float64
 4   3       291813 non-null  float64
 5   4       291813 non-null  float64
 6   5       291813 non-null  float64
 7   6       291813 non-null  float64
 8   7       291813 non-null  float64
 9   8       291813 non-null  float64
 10  9       291813 non-null  float64
 11  10      291813 non-null  float64
 12  11      291813 non-null  float64
 13  12      291813 non-null  float64
 14  13      291813 non-null  float64
 15  14      291813 non-null  float64
 16  15      291813 non-null  float64
 17  16      291813 non-null  float64
 18  17      291813 non-null  float64
 19  18      291813 non-null  float64
 20  19      291813 non-null  float64
 21  20      29

In [9]:
look_on(train)

,Id,0,1,2,3,4,5,6,7,8,...,63,64,65,66,67,68,69,70,71,Target
0,109249-query,-24.021454,3.122524,-80.947525,-112.329994,191.090180,-66.903130,-759.626065,-75.284454,120.55149,...,-24.601670,-167.760770,133.678516,68.184600,26.317545,11.938202,148.549320,-778.563381,-46.877750,66971-base
1,34137-query,-82.033580,8.115866,-8.793022,-182.972100,56.645336,-52.597610,-55.720337,130.059250,129.38335,...,54.448433,-120.894806,-12.292085,66.608116,-27.997612,10.091335,95.809265,-1022.691531,-88.564705,1433819-base
2,136121-query,-75.719640,-0.223386,-86.186130,-162.064060,114.320114,-53.394600,-117.261013,-24.857851,124.80780,...,-5.609123,-93.029880,-80.997871,63.733383,11.378683,62.932007,130.975390,-1074.464888,-74.861176,290133-base
3,105191-query,-56.580620,5.093593,-46.943110,-149.039120,112.436430,-76.820510,-324.995645,-32.833107,119.47865,...,21.624313,-158.880370,179.597294,69.891360,-33.804955,233.914610,122.868546,-1074.464888,-93.775375,1270048-base
4,63983-query,-52.725650,9.027046,-92.829650,-113.111010,134.124970,-42.423073,-759.626065,8.261169,119.49023,...,13.807772,-208.650040,41.742014,66.522420,41.362930,162.723050,111.261310,-151.162805,-33.831450,168591-base


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 74 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      9999 non-null   object 
 1   0       9999 non-null   float64
 2   1       9999 non-null   float64
 3   2       9999 non-null   float64
 4   3       9999 non-null   float64
 5   4       9999 non-null   float64
 6   5       9999 non-null   float64
 7   6       9999 non-null   float64
 8   7       9999 non-null   float64
 9   8       9999 non-null   float64
 10  9       9999 non-null   float64
 11  10      9999 non-null   float64
 12  11      9999 non-null   float64
 13  12      9999 non-null   float64
 14  13      9999 non-null   float64
 15  14      9999 non-null   float64
 16  15      9999 non-null   float64
 17  16      9999 non-null   float64
 18  17      9999 non-null   float64
 19  18      9999 non-null   float64
 20  19      9999 non-null   float64
 21  20      9999 non-null   float64
 22  

In [10]:
look_on(validation)

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,196680-query,-59.383420,8.563436,-28.203072,-134.22534,82.73661,-150.572170,-129.178969,23.670555,125.66636,...,-103.481630,79.564530,-120.31357,54.218155,68.50073,32.681908,84.19686,136.412960,-1074.464888,-21.233612
1,134615-query,-103.912150,9.742726,-15.209915,-116.37310,137.69880,-85.530075,-776.123158,44.481530,114.67121,...,-51.193770,49.299644,-101.89454,105.560548,67.80104,13.633057,108.05138,111.864456,-841.022331,-76.567980
2,82675-query,-117.923280,-3.504554,-64.299390,-155.18713,156.82137,-34.082264,-537.423653,54.078613,121.97396,...,-115.176155,48.636130,-132.17967,-0.988696,68.11125,107.065216,134.61765,134.080000,27.773269,-32.401714
3,162076-query,-90.880554,4.888542,-39.647797,-131.75010,62.36212,-105.593270,-347.132493,-83.351750,133.91331,...,-112.293790,54.884007,-177.56935,-116.374997,67.88766,136.893980,124.89447,117.707750,-566.343980,-90.905556
4,23069-query,-66.946740,10.562773,-73.781830,-149.39787,2.93866,-51.288853,-587.189361,-2.764402,126.56105,...,-116.440605,47.279976,-162.65400,107.409409,67.78526,-60.976490,142.68571,82.264300,-345.340457,-48.572525


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 73 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      10000 non-null  object 
 1   0       10000 non-null  float64
 2   1       10000 non-null  float64
 3   2       10000 non-null  float64
 4   3       10000 non-null  float64
 5   4       10000 non-null  float64
 6   5       10000 non-null  float64
 7   6       10000 non-null  float64
 8   7       10000 non-null  float64
 9   8       10000 non-null  float64
 10  9       10000 non-null  float64
 11  10      10000 non-null  float64
 12  11      10000 non-null  float64
 13  12      10000 non-null  float64
 14  13      10000 non-null  float64
 15  14      10000 non-null  float64
 16  15      10000 non-null  float64
 17  16      10000 non-null  float64
 18  17      10000 non-null  float64
 19  18      10000 non-null  float64
 20  19      10000 non-null  float64
 21  20      10000 non-null  float64
 22 

In [11]:
look_on(validation_answer)

,Id,Expected
0,196680-query,1087368-base
1,134615-query,849674-base
2,82675-query,4183486-base
3,162076-query,2879258-base
4,23069-query,615229-base


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        10000 non-null  object
 1   Expected  10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [16]:
train.Target.value_counts()

803095-base     3
804781-base     3
272609-base     3
839040-base     3
79670-base      3
               ..
2222522-base    1
24444-base      1
501422-base     1
826204-base     1
505720-base     1
Name: Target, Length: 9672, dtype: int64

In [ ]:
12
34
56

## 2.3 EDA

Несмотря на то, что данные обезличены, EDA здесь также будет полезен: все ли столбцы имеют одинаковое распределение значений? Есть ли столбцы, которые для модели были бы мало полезны? Есть ли сильно скоррелированные друг с другом столбцы? Может быть, есть смысл на первом этапе подавать в модель не все фичи, а наиболее информативные? Есть ли пропуски? Явные дубликаты? Если есть - что с ними делать? Есть ли аномалии в распределениях? Следующий важный вопрос - не требуется ли масштабирование данных? Ответить на этот вопрос можно, например, замерив метрику с масштабированием и без масштабирования признаков.

## 2.4 Целевая метрика

Наша целевая метрика - accuracy@n. Собственно, что это такое. Вспомним, что 

$$
Accuracy = \frac{Correct\ predictions}{All\ predictions}
$$

Представим расчет метрики в цикле, перебирая все предложенные моделью ответы. При этом каждое предсказание содержит в себе не 1 ответ, а сразу n, и если среди предложенных вариантов окажется правильный - числитель и знаменатель увеличиваются на 1. А если нет ни одного - то на 1 увеличивается только знаменатель. В нашей задаче n = 5. Хорошо бы добиться accuracy@5 ≥ 0,7. Кстати, легко заметить, что accuracy@1 - это самая обычная accuracy.

## 📊 Create FAISS [index](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes) for small dataset


[Guideline](https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index)

Hint: Use numpy [ascontigiousarray](https://numpy.org/doc/stable/reference/generated/numpy.ascontiguousarray.html) - object which is stored in one [unbroken block](https://www.educative.io/answers/what-is-the-numpyascontiguousarray-function-in-python) in memory -  to load vectors in FAISS

In [ ]:
dims = df_base.shape[1]
n_cells = 20
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

In [ ]:
%%time
idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))

In [ ]:
base_index = {k: v for k, v in enumerate(df_base.index.to_list())}

## 🔍 Search

In [ ]:
targets = df_train["Target"]
df_train.drop("Target", axis=1, inplace=True)

In [ ]:
%%time
candidate_number = 5
r, idx = idx_l2.search(np.ascontiguousarray(df_train.values).astype('float32'), candidate_number)

## 📈 Accuracy@candidate_number calculation

In [ ]:
acc = 0
for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])
print(f'Accuracy @ {candidate_number} = {acc / len(idx):.1%}')

In [ ]:
## ❓❓❓ What's next?

For full dataset it is strongly recommended to test your code on the small batch before loading all dataset to FAISS

You can make your own research:
- change number of cells
- change number of candidates
- change indexes
- add another ML models to improve the FAISS result
- change the accelerator: Hint: Search method on GPU differs a bit from the similar method on CPU
-.....

Remember, that in Colab you have only 12 GB of RAM, so remove variables and objects if necessary

**Good Luck!**